<a href="https://colab.research.google.com/github/ashishkhareiitr/projects/blob/master/FinalPoseDetectionModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Capstone Project - Yoga Pose Detection 

     by Ashish Khare , Amalendu Tiwari , Harshada Khanvilkar and Divya Tripathi**

## ***# Table of Contents***

>[Capstone Project - Yoga Pose Detection by Ashish Khare , Amalendu Tiwari , Harshada Khanvilkar and Divya Tripathi](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=FWL9nmh1u1bj)

>>[# Table of Contents](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=av03T_y6usNx)

>[Load Yolo Configuration](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=w_-ezk6Bqu03)

>[Load Pacakges](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=KLW5csvls_zm)

>[Function : Get Output Layers](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=uCvleoqytQtd)

>[Function : Draw Prediction](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=yUR830lzvOUe)

>[Function : Load Images](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=OsXqj1-3ydoa)

>[Function : Canny Edge](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=VMDjAXtfvhYV)

>[Function : Calculate key frame thresold](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=hq0HaAITjnmy)

>[Function : Split Frames](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=HlJXFUSZjs5L)

>[Function : Train Data Prep](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=XBnFSTduy1p2)

>[Function : Test Data Prep](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=qUY5FpWoy9LS)

>[Function : Yolo object detection and crop human](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=Z6v0EV1NkBUe)

>>[Please make sure yolov3.cfg , yolov3.txt and yolov3.weights are uploaded](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=Z6v0EV1NkBUe)

>[CNN training and Accuracy](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=pmpXUP_TzIwT)

>[Running test/train data labels and CNN](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=szNGmlrLzV7f)

>[Predict using saved model](#updateTitle=true&folderId=1Iapkl62Ujq_fuCjoQQqCFUtdpPIw19Ng&scrollTo=t-Lo_rRUzvB9)



# Load Yolo Configuration

In [0]:
!wget https://pjreddie.com/media/files/yolov3.weights

# please also upload yolov3.txt and yolov3.cfg  

# Load Pacakges

In [1]:
from __future__ import print_function
import cv2
import argparse
import sys
import PIL.Image
import os
from sys import platform
import re
from os import listdir
import ntpath #for retrieving filename
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
import pandas as pd
from keras.utils import np_utils
from keras import backend as K
from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pylab as pl
import matplotlib.pyplot as plt
from statistics import stdev 
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from keras.preprocessing import image

Using TensorFlow backend.


# Function : Get Output Layers

---



In [0]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# Function : Draw Prediction

In [0]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    image = cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return image

# Function : Load Images

In [0]:
_nsre = re.compile('([0-9]+)')

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(_nsre, s)]  
  
def loadImages(path):
    # return array of images
    
    imagesList = listdir(path)
    imagesList.sort(key=natural_sort_key)
    #imagesList = imagesList.sort()     
    loadedImages = []
    for image in imagesList:
      if image is not None:
        img = PIL.Image.open(path + image)
        loadedImages.append(img)
      else:
        print('not loaded')
    
       
    return loadedImages

# Function : Canny Edge

In [0]:
############canny edge detection############
 
def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
 
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
 
	# return the edged image
	return edged

# Function : Calculate key frame thresold

In [0]:
##############Key Frames thresold ##########################
def KeyFrameThreshold(VideoPath):

    print('calculating key frame thresold.....')
    fgbg = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=25, detectShadows=True)
    cap = cv2.VideoCapture(VideoPath)
    # Read the first frame.
    ret, prev_frame = cap.read()
    prev_edges = auto_canny(prev_frame)
    storediff =[]
    
   
    while ret:
        ret, curr_frame = cap.read()
        
        if ret:
               
            gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (3, 3), 0)
            
            curr_frame_bg = fgbg.apply(blurred)
            #curr_edges = cv2.Canny(curr_frame_bg,100,200)
            curr_edges = auto_canny(curr_frame_bg)
            
            diff = cv2.absdiff(curr_edges, prev_edges)
            non_zero_count = np.count_nonzero(diff)
            storediff.append(non_zero_count)
            #icount = icount +1
            #print(non_zero_count)
            prev_edges = curr_edges
                           
            #print(diff)
            #meandiff = meandiff + diff
            #print(meandiff)
            #cv2.imshow('Original',frame) 
      
            # finds edges in the input image image and 
            # marks them in the output map edges 
            #edges = cv2.Canny(frame,100,200) 
      
            # Display edges in a frame 
            #cv2.imshow('Edges',edges) 
            #storediff.append(diff)
    meanstorediff =  np.mean(storediff)
    stdstorediff =   np.std(storediff)
    min_p_frame_thresh = meanstorediff-1.5*stdstorediff
    max_p_frame_thresh = meanstorediff+1.5*stdstorediff 
    
    print('completed calculating key frame thresold.')
    
    return min_p_frame_thresh,max_p_frame_thresh


# Function : Split Frames

> Read videos from folder and store as keyframes



In [0]:
#########################Extracting and saving key frames################################
            
def KeyFrame(VideoPath,KeyFrameFolder,minThreshold,maxThreshold):

    print('extracting key frames.....')
    cap = cv2.VideoCapture(VideoPath)
    # Read the first frame.
    ret, prev_frame = cap.read()
    #prev_frame = cv2.resize(prev_frame, (128, 128))    
    prev_edges = auto_canny(prev_frame)
    
    #prev_edges = cv2.Canny(prev_frame,100,200)
    count = 0
    vfolder = KeyFrameFolder
    os.mkdir(vfolder)
    
    fgbg = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=25, detectShadows=True)
    #scaling_factor = 1.5
    while ret:
      ret, curr_frame = cap.read()
      #curr_frame = cv2.resize(curr_frame, (128, 128)) 
      #curr_frame = cv2.resize(curr_frame, None, fx=scaling_factor, fy=scaling_factor,interpolation=cv2.INTER_AREA)
      if ret:
        gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (3, 3), 0)
        curr_frame_bg = fgbg.apply(blurred)
        curr_edges = auto_canny(curr_frame_bg)
           
        diff = cv2.absdiff(curr_edges, prev_edges)
        non_zero_count = np.count_nonzero(diff)
        #print(non_zero_count)
          
        if non_zero_count > maxThreshold:
          print('Writing Key-Frame...')
          count = count+1
          cv2.imwrite(os.path.join(vfolder,"frame{:d}.jpg".format(count)), curr_frame) 
          #cv2.imshow('Original',curr_frame) 
          prev_edges = curr_edges
    print('completed extracting key frames.')    
    return count

# Function : Train Data Prep

In [0]:
######################################### train data prep work #####################

def TrainDataPrep(TrainDataPath,img_row,img_col):

    train_data = []
    train_labels_one_hot =[]
    trainfileList =[]
    
    for root, dirs, files in os.walk(TrainDataPath):
        for file in files:
            if file.endswith(".jpg"):
                 fullName = os.path.join(root, file)
                 trainfileList.append(fullName)
    
    fgbg = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=25, detectShadows=True) 
    for imagePath in trainfileList:
        
        image = cv2.imread(imagePath)
        if image is not None:
            
            image = cv2.resize(image, (img_row,img_col))
            #image = cv2.resize(image, (128, 128))
            
            #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            #blurred = cv2.GaussianBlur(gray, (3, 3), 0)
            #curr_frame_bg = fgbg.apply(blurred)
            #image = auto_canny(curr_frame_bg)
            
            image = img_to_array(image)
            #x = x.reshape((1,) + x.shape)
            train_data.append(image)
            train_data[0].shape
            path = os.path.split(imagePath)
            x = path[0]
            #x = re.sub(r"\\",r"\\\\", x)
            #print(x)
            label = x.split("/")[2]
            label = int(label)
            train_labels_one_hot.append(label)
            print(imagePath)
            print(label)
            
        else:
                print(imagePath,"image  not loaded")
        
    train_data = np.array(train_data)
    train_labels_one_hot = np.array(train_labels_one_hot)
    print('train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    
    return train_data,train_labels_one_hot


# Function : Test Data Prep

In [0]:
#################################test data prep work ###############################################3

def TestDataPrep(TestDataPath,img_row,img_col):

    test_data = []
    test_labels_one_hot =[]
    testfileList =[]
    
    for root, dirs, files in os.walk(TestDataPath):
        for file in files:
            if file.endswith(".jpg"):
                 fullName = os.path.join(root, file)
                 testfileList.append(fullName)
    
    fgbg = cv2.createBackgroundSubtractorMOG2(history=20, varThreshold=25, detectShadows=True)
    for imagePath in testfileList:
        
        image = cv2.imread(imagePath)
        if image is not None:
            
            image = cv2.resize(image, (img_row,img_col))
            #image = cv2.resize(image, (128, 128))
                
            #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            #blurred = cv2.GaussianBlur(gray, (3, 3), 0)
            #curr_frame_bg = fgbg.apply(blurred)
            #image = auto_canny(curr_frame_bg)
            
            image = img_to_array(image)
            #x = x.reshape((1,) + x.shape)
            test_data.append(image)
            test_data[0].shape
            path = os.path.split(imagePath)
            x = path[0]
            label = x.split("/")[2]
            label = int(label)
            test_labels_one_hot.append(label)
            print(imagePath)
            print(label)
            
        else:
                print(imagePath,"image not loaded")
        
    
    test_data = np.array(test_data)
    test_labels_one_hot = np.array(test_labels_one_hot)
    print('test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')  
    
    return test_data,test_labels_one_hot



# Function : Yolo object detection and crop human

## > # Please make sure yolov3.cfg , yolov3.txt and yolov3.weights are uploaded



In [0]:
path_loadImages =  "backup data/test/2/"
count = 0


imgs = loadImages(path_loadImages)

#fgbg = cv2.createBackgroundSubtractorMOG2(history=15, varThreshold=25, detectShadows=False) 

for i in range(1,len(imgs)):
  
  print(imgs[i-1].filename)

  image=cv2.imread(imgs[i-1].filename)

#image = cv2.imread("frame328.jpg")

  Width = image.shape[1]
  Height = image.shape[0]
  scale = 0.00392

  classes = None

  with open("yolov3.txt", 'r') as f:
    
    classes = [line.strip() for line in f.readlines()]

  COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

  weights = "yolov3.weights"
  config = "yolov3.cfg"

  net = cv2.dnn.readNet(weights, config)

  blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

  net.setInput(blob)

  outs = net.forward(get_output_layers(net))

  class_ids = []
  confidences = []
  boxes = []
  conf_threshold = 0.5
  nms_threshold = 0.4


  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.5:
              center_x = int(detection[0] * Width)
              center_y = int(detection[1] * Height)
              w = int(detection[2] * Width)
              h = int(detection[3] * Height)
              x = center_x - w / 2
              y = center_y - h / 2
              class_ids.append(class_id)
              confidences.append(float(confidence))
              boxes.append([x, y, w, h])


  indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

  for i in indices:
      i = i[0]
      box = boxes[i]
      x = box[0]
      y = box[1]
      w = box[2]
      h = box[3]
      draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

  #cv2.imshow("object detection", image)
  #cv2.waitKey()

  #cv2.imwrite("object-detection.jpg", image)
  #cv2.destroyAllWindows()
  cv2.imwrite(os.path.join(os.getcwd(),"2test_yolo{:d}.jpg".format(count)),image )
  
  xx = int(x)
  yy = int(y)
  ww = int(w)
  hh = int(h)
  
  img = cv2.imread("2test_yolo{:d}.jpg".format(count))
  print(xx,yy,ww,hh)
  
  if img is not None:
    
    
    crop_img = img[yy:yy+hh,xx:xx+ww]
  #crop_img = img[604:700,420:900]
  #y:y+h, x:x+w]
  #cv2.imshow("cropped", crop_img)
    cv2.imwrite(os.path.join(os.getcwd(),"2test_cropyolo{:d}.jpg".format(count)),crop_img)
  else:
    print("not cropped")
    
  
  cropped_img = cv2.imread("2test_cropyolo{:d}.jpg".format(count))
  
  if cropped_img is not None:
    
    gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
  else:
    print("not able to convert to gray")
    
  #blurred = cv2.GaussianBlur(gray, (3, 3), 0)
  #curr_frame_bg = fgbg.apply(cropped_img)
  
  cv2.imwrite(os.path.join(os.getcwd(),"2test_CropBGyolo{:d}.jpg".format(count)),gray)
  
  count = count +1
  

# CNN training and Accuracy

In [0]:
########################CNN model################################################################

def YogaCNNModel(train_data,train_labels_one_hot,test_data,test_labels_one_hot,save_dir,model_name,num_classes,batch_size,epochs,img_row,img_col):
    
    spe = int( np.ceil(train_data.shape[0] / batch_size) )
    
    print('train samples',train_data.shape)
    print('test samples',test_data.shape)
    print('train labels',train_labels_one_hot.shape)
    print('test labels',test_labels_one_hot.shape)
    
    train_labels_one_hot = np_utils.to_categorical(train_labels_one_hot)
    test_labels_one_hot = np_utils.to_categorical(test_labels_one_hot)
    

     
    model = Sequential()
    model.add(Conv2D(32, (3,3), padding='same',input_shape=(img_row,img_col,3)))
    print('applying first CNN layer')
     
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    
    print('aplying second CNN layer')
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    print('applying third CNN layer')
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same'))
    print('applying fourth CNN layer')
    model.add(Activation('relu'))
    
    #model.add(Conv2D(64, (3, 3)))
    
    #print('applying fivth CNN layer')
    #model.add(Activation('relu'))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(128))
    #print('applying sixth CNN layer')
    model.add(Activation('relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    print('applying seventh CNN layer')
    model.add(Activation('softmax'))
    
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    
    # Let's train the model using RMSprop
    model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
    
    #train_data = train_data.astype('float32')
    #test_data = test_data.astype('float32')
    #train_data /= 255
    #test_data /= 255
    
    if not data_augmentation:
        print('Not using data augmentation.')
        print('fitting CNN model without data augmentation')
        model.fit(train_data, train_labels_one_hot,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(test_data, test_labels_one_hot),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        print('fitting CNN model with data augmentation')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            #steps_per_epoch = 1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)
    
        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(train_data)
    
        # Fit the model on the batches generated by datagen.flow(). 
        #steps_per_epoch should be equivalent to the total number of samples divided by the batch size.
        model.fit_generator(datagen.flow(train_data, train_labels_one_hot,
                                         batch_size=batch_size),
                            epochs=epochs,steps_per_epoch=spe,
                            validation_data=(test_data, test_labels_one_hot))
    
    # Save model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
    
    # Score trained model.
    scores = model.evaluate(test_data, test_labels_one_hot, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    
    #model.save("CBA-model.h5")
    print('Trained on following stastics:')
    print('train samples',train_data.shape)
    print('test samples',test_data.shape)
    print('train labels',train_labels_one_hot.shape)
    print('test labels',test_labels_one_hot.shape)

    return scores[0],scores[1] 

                                #FUNCTION MODULES end#


# Running test/train data labels and CNN

In [0]:
# testing
train_data,train_labels_one_hot = TrainDataPrep("yogaposes/train",128,128) 
# testing
test_data,test_labels_one_hot = TestDataPrep("yogaposes/test",128,128) 

In [0]:
data_augmentation = True

YogaCNNModel(train_data,train_labels_one_hot,test_data,test_labels_one_hot,"finalmodel","finalmodel.h5",3,1,100,128,128)

# Predict using saved model

In [0]:
def PredictClass(modelPath,validationImagesPath):

   
    # dimensions of our images
    img_width, img_height = 128, 128
    
    # load the model we saved
    model = load_model('finalmodel.h5')
    model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    
    # predicting images
    img = image.load_img('frame14.jpg', target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    print(classes)
    
    #predictions = classifier.predict(images)

    #print(predictions)
    
    # predicting multiple images at once
#    img = image.load_img('test2.jpg', target_size=(img_width, img_height))
#    y = image.img_to_array(img)
#    y = np.expand_dims(y, axis=0)
#    
#    # pass the list of multiple images np.vstack()
#    images = np.vstack([x, y])
#    classes = model.predict_classes(images, batch_size=10)
#    
#    # print the classes, the images belong to
#    print(classes)
#    print(classes[0])
#    print(classes[0][0])
    
    return classes